## <b><font color='darkblue'>OpenAI 免費開源語音辨識系統-- Whisper 安裝簡介及原理</font></b>
([article source](https://ithelp.ithome.com.tw/articles/10311957)) <b><font size='3ptx'>[Whisper](https://github.com/openai/whisper/tree/main) 是 OpenAI 提供的一種開源的自動語音辨識( <font color='brown'>Automatic Speech Recognition，ASR</font> )的神經網路模型，用來執行語音辨識(language identification)與翻譯(speech translation)的功能。能夠將各種語言的語音轉錄成文字(multilingual speech recognition)，甚至可以處理較差的音頻品質或過多的背景雜訊。</font></b>

模型架構，安裝需求和標準，可參考 [官方文檔](https://github.com/openai/whisper/tree/main): 
* Introducing Whisper - OpenAI: https://openai.com/research/whisper (裡面就有"Read paper", "View code", "View model card")
* Read paper: https://cdn.openai.com/papers/whisper.pdf
* View code: https://github.com/openai/whisper
* View model card:https://github.com/openai/whisper/blob/main/model-card.md

![arch](https://ithelp.ithome.com.tw/upload/images/20230417/20157739ZWCLSX2P98.png)

上圖為 Whisper 的模型訓練方法,可以從 paper 看,不過筆者在這邊簡單解釋一下:
1. <b>數據資料集(data)</b>: 左上為 Multitask(多工) training data ,總共 680k hours (其實也透漏資料大小), 為何叫多工呢? 因為它分為四種: (1) 英文轉錄, (2) 任何語言翻譯為英文,(3) 非英語系轉錄,(4) 不講話(只有背景音樂)
2. <b>訓練 (training)</b>:它的訓練過程在右邊,主要為 sequence-to-sequence learning, sequence-to-sequence model 是一類將序列轉化為序列技術的統稱，最初即是由 Google Translate 研究團隊所開發。關於 sequence-to-sequence, 主要是由 Encoder 和 Decoder 兩個 RNN 所構成, 可以想像 Encoder 相當於處理 input data, 我們餵資料給它,經由特徵工程(或 attention, 或影像壓縮處理, 去噪....等等特殊處理),然後再經由 Decoder 之後輸出。關於 sequence-to-sequence,可以參考這篇: http://zake7749.github.io/2017/09/28/Sequence-to-Sequence-tutorial/ ,寫的還不錯。
3. **Multitask training format**: 最下方為 Multitask training format,剛剛有談到拿來作為訓練資料集,主要分為三種類型的 token: special tokens, text tokens, timestamp tokens; 一開始,輸入原本的語音內容,先將內容分為兩類:有語音,與沒有語音(可能沒錄好,或是正好為講話間隙,即長時間沒有語音出現);沒有語音則經由語音活性檢測 (Voice activity detection，VAD),再度確認,或是將其處理掉;若有語音則分為英語系,和非英語系兩種,都可由下列兩種方式錄製或翻譯:
  * 需要時間校準(Time-aligned Transcription): 於 text token 之間,插入 begin time 和 end time。所以您可以看到之後我下面 output 錄製語音的 .txt 檔案,有開始,和結束時間紀錄。
  * 不需時間校準 (Text-only Transcription): 只有 text tokens,但它可以允許 dataset fine tune。


好了,所以我們可以看到圖片的左下方有個箭頭,將 Multitask training data (多工訓練資料) 餵進去 Log-Mel Spectrogram (Mel Spectrogram, 梅爾頻譜), 何謂 <b><font color='darkblue'>梅爾頻譜</font>? 其實,它就是在語音處理中常常用到的頻譜,在語音分類中把信號變成圖片,然後用分類圖片的算法</b> (例如: CNN) <b>來分類語音</b>。更多詳細梅爾頻譜的資料,請參考: https://blog.csdn.net/bo17244504/article/details/124707265
https://medium.com/analytics-vidhya/understanding-the-mel-spectrogram-fca2afa2ce53
https://zhuanlan.zhihu.com/p/351956040

因此, 這些頻譜經由 CNN (2xConv1D + GELU) 的方法 training (訓練),再經由正弦波位置編碼(Sinusoidal position encoder, 所謂正弦波位置編碼,簡單講就是在 transformer 架構中加入一些關於 token 位置的信息,其演算法使用了不同频率的正弦函數来作為位置編碼,所以叫作正弦波位置編碼),接著,進入 Tranformer Encoder 的過程 (MLP + self-attention),(MLP + self-attention)...(MLP + self-attention),(self-attention 來自相同序列),幾層我不知道,或許多層多次,然後再經由 cross attention (cross attention 來自不同序列),經由 transformer decoder 輸出。當然,在訓練過程中,我們可以得知 Multitask training format 中 token 的信息,與訓練好的模型來預測下一個 token。這也意味著可以經由模型來輸出講者的說話模式(內容)。

* **正弦波位置編碼可參考**: https://zhuanlan.zhihu.com/p/98641990, https://zhuanlan.zhihu.com/p/338592312
* **MLP**:https://chih-sheng-huang821.medium.com/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF-%E5%A4%9A%E5%B1%A4%E6%84%9F%E7%9F%A5%E6%A9%9F-multilayer-perceptron-mlp-%E5%90%AB%E8%A9%B3%E7%B4%B0%E6%8E%A8%E5%B0%8E-ee4f3d5d1b41
https://zh.wikipedia.org/zh-tw/%E5%A4%9A%E5%B1%82%E6%84%9F%E7%9F%A5%E5%99%A8

## <b><font color='darkblue'>Installation</font></b>
以上模型講解完畢,接下來是安裝的部分. 如果您已有 OpenAI ChatGPT API key ([reference](https://platform.openai.com/docs/quickstart/step-2-setup-your-api-key))，則可以直接進行 OpenAI 語音辨識，不需要額外安裝 Whisper. 另外在你個人帳號中, 請將你的 organization ID 設定到環境變數 `OPENAI_ORG_ID`.

In [12]:
import os

In [13]:
if 'OPENAI_API_KEY' not in os.environ or 'OPENAI_ORG_ID' not in os.environ:
  raise Exception('Please prepare API key for OpenAPI call')

In [3]:
#!pip install openai  #安裝套件

In [14]:
import os
import openai
openai.organization = os.getenv('OPENAI_ORG_ID')
openai.api_key = os.getenv("OPENAI_API_KEY")

看 Moedl 詳細資訊及權限:

In [5]:
#openai.Model.list()

但如果您沒有OpenAI ChatGPT API key，或您對於 Whisper 很感興趣，則可以參考以下步驟,也就是本文的重點 ([how-to](https://ithelp.ithome.com.tw/articles/10311957))

## <b><font color='darkblue'>Speech to text API Usages</font></b>
([article source](https://platform.openai.com/docs/guides/speech-to-text)) The Audio API provides two speech to text endpoints, `transcriptions` and `translations`, based on our state-of-the-art open source large-v2 [**Whisper model**](https://openai.com/blog/whisper/). They can be used to:
* Transcribe audio into whatever language the audio is in.
* Translate and transcribe the audio into english.

File uploads are currently limited to 25 MB and the following input file types are supported: `mp3`, `mp4`, `mpeg`, `mpga`, `m4a`, `wav`, and `webm`.

### <b><font color='darkgreen'>Quickstart</font>

In [6]:
TEST_AUDIO_FILE = '../../medium/Easy-speech-to-text-with-python/datas/OSR_us_000_0010_8k.wav'

#### <b><font size='3ptx'>Transcriptions</font></b>
The transcriptions API takes as input the audio file you want to transcribe and the desired output file format for the transcription of the audio. We currently support multiple input and output file formats.

In [15]:
import openai
from packaging import version

required_version = version.parse("1.1.1")
current_version = version.parse(openai.__version__)
print(f'Current openai version is {current_version}')
if current_version < required_version:
    raise ValueError(f"Error: OpenAI version {openai.__version__}"
                     " is less than the required version 1.1.1")
else:
    print("OpenAI version is compatible.")

# -- Now we can get to it
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # should use env variable OPENAI_API_KEY

Current openai version is 1.6.1
OpenAI version is compatible.


In [8]:
audio_file= open(TEST_AUDIO_FILE, "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)

In [9]:
transcript

Transcription(text='The birch canoe slid on the smooth planks. Glued the sheet to the dark blue background. It is easy to tell the depth of a well. These days a chicken leg is a rare dish. Rice is often served in round bowls. The juice of lemons makes fine punch. The box was thrown beside the park truck. The hogs were fed chopped corn and garbage. Four hours of steady work faced us. A large size in stockings is hard to sell.')

The Audio API also allows you to set additional parameters in a request. For example, if you want to set the <font color='violet'>response_format</font> as text, your request would look like the following:

In [10]:
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)

In [11]:
transcript

'The birch canoe slid on the smooth planks. Glued the sheet to the dark blue background. It is easy to tell the depth of a well. These days a chicken leg is a rare dish. Rice is often served in round bowls. The juice of lemons makes fine punch. The box was thrown beside the park truck. The hogs were fed chopped corn and garbage. Four hours of steady work faced us. A large size in stockings is hard to sell.\n'

## <b><font color='darkblue'>End to end testing</font></b>
Here let's do an end-to-end testing:
1. Make a voice recording and save as wav file
2. Feed the wav file into whisper API
3. Show the transformed text

### <b><font color='darkgreen'>Make a voice recording</font></b>

In [3]:
import pyaudio
import wave

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 512
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "recordedFile.wav"

In [4]:
audio = pyaudio.PyAudio()

ALSA lib setup.c:547:(add_elem) Cannot obtain info for CTL elem (MIXER,'AC97 2ch->4ch Copy Switch',0,0,0): No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib setup.c:547:(add_elem) Cannot obtain info for CTL elem (MIXER,'AC97 2ch->4ch Copy Switch',0,0,0): No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib setup.c:547:(add_elem) Cannot obtain info for CTL elem (PCM,'IEC958 Playback PCM Stream',0,0,0): No such file or di

In [5]:
print("----------------------record device list---------------------")
info = audio.get_host_api_info_by_index(0)

----------------------record device list---------------------


In [6]:
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  Ensoniq AudioPCI: ES1371 DAC2/ADC (hw:0,0)
Input Device id  2  -  sysdefault
Input Device id  4  -  samplerate
Input Device id  5  -  speexrate
Input Device id  6  -  upmix
Input Device id  7  -  vdownmix
Input Device id  8  -  default


In [7]:
index = int(input())
print("recording via index "+str(index))

stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,input_device_index = index,
                frames_per_buffer=CHUNK)
print ("recording started")
Recordframes = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    Recordframes.append(data)
print ("recording stopped")
 
stream.stop_stream()
stream.close()
audio.terminate()

0
recording via index 0
recording started
recording stopped


In [8]:
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(Recordframes))
waveFile.close()

### <b><font color='darkgreen'>Extract text from wave file</font></b>
這邊我準備一個 local 錄的檔案 `wave_files/test.wav`:

In [9]:
test_wav_file = 'wave_files/test.wav'

In [17]:
os.path.isfile(test_wav_file)

True

In [18]:
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=open(test_wav_file, "rb"),
  response_format="text",
)

In [19]:
transcript

'Hello. Hello. This is for testing.\n'

轉換的結果的確如我的發音.

## <b><font color='darkblue'>Supplement</font></b>
* [YT - How to record microphone input in Python with PyAudio](https://www.youtube.com/watch?v=9GJ6XeB-vMg)
* [Ubuntu install pyaudio](https://blog.csdn.net/weixin_38369492/article/details/125992255)
```shell
$ apt-get install portaudio19-dev
$ apt-get install python3-all-dev
$  pip install pyaudio
```
* [M4A to WAV converter](https://cloudconvert.com/m4a-to-wav)